## Tutorial 1: Phase diagram

Information about compounds will be taken from MaterialsProject database https://next-gen.materialsproject.org

Firstly, we should import necessary python libraries

In [ ]:
# !pip install thermochem

In [ ]:
#for Matproject
from mp_api.client import MPRester
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from pymatgen.entries.mixing_scheme import MaterialsProjectDFTMixingScheme

#library for working with tables
from pandas import DataFrame

import collections

import numpy as np
import matplotlib.pyplot as plt

# !pip install mpcontribs-client #if not installed and show error

The interaction with Materials Project will occur from this Notebook via API. 

API requires a special key, which should be generated on their webpage https://next-gen.materialsproject.org/api

In [ ]:
your_api_key = "Jx8eNRBN0gQyptvUkzEbjCd9kUArLvt0" #put here your own API key from https://next-gen.materialsproject.org/api

## 1. Binary phase diagram (Convex hull)

Here we construct a binary phase diagram in a format of Convex hull for the system of Co-O.



**Step 1: obtain information about all the Co_xO_y compounds in Materials Project database**

In [ ]:
with MPRester(your_api_key) as mpr:
    # Obtain only corrected GGA and GGA+U ComputedStructureEntry objects
    entries_co_o = mpr.get_entries_in_chemsys(elements=["Co",  "O"], 
                                         additional_criteria={"thermo_types": ["GGA_GGA+U"]}
                                         ) 

print('Done')

In [ ]:
# What information does the entry contain?
print(entries_co_o[50])

**Step 2: Simple way to construct Convex Hull**

In [ ]:
pd_co_o = PhaseDiagram(entries_co_o)

plotter = PDPlotter(pd_co_o, show_unstable=True) #you can switch off unstable compounds via plot interface
plotter.show()

**Step 3: analysis**

In [ ]:
#We can write all found compounds into a table
data = collections.defaultdict(list) #create empty dictionary

#fill the dictionary by necessary properties
# print(dir(entries_co_o[0])) #to show all possible properties of entries_co_o[0] object
for e in entries_co_o:
    data["Materials ID"].append(e.entry_id)
    data["Composition"].append(e.composition.reduced_formula)
    data["Spacegroup"].append(e.structure.get_space_group_info())
    data["Ehull"].append(pd_co_o.get_e_above_hull(e))    

#convert dictionary to more usable format using pandas
df = DataFrame(data, columns=["Materials ID", "Spacegroup", "Composition", "Ehull"])
print(df)

#write all the compounds into co-o_all.csv file
df.to_csv('co-o_all.csv') 

Show only stable compounds use the pd.stable_entries

In [ ]:
data = collections.defaultdict(list) #create empty dictionary

#fill the dictionary by necessary properties
for e in pd_co_o.stable_entries:
    data["Materials ID"].append(e.entry_id)
    data["Composition"].append(e.composition.reduced_formula)
    data["Spacegroup"].append(e.structure.get_space_group_info())
    data["Ehull"].append(pd_co_o.get_e_above_hull(e))    

df = DataFrame(data, columns=["Materials ID", "Spacegroup", "Composition", "Ehull"])
print(df)
#write only stable compounds into co-o_stable.csv file
df.to_csv('co-o_stable.csv') 

We obtained N stable compounds characterized by Ehull or energy above hull = 0, that means they belong to the convex hull line

## 2. Ternary phase diagram (Convex Hull)

Ternary phase diagram construction has the same procedure:

In [ ]:
with MPRester(your_api_key) as mpr:

    # Obtain GGA, and GGA+U ComputedStructureEntry objects
    entries_li_co_o = mpr.get_entries_in_chemsys(elements=["Li", "Co", "O"], 
                                         additional_criteria={"thermo_types": ["GGA_GGA+U"]}) 
    
   
    # Construct phase diagram
    pd_li_co_o = PhaseDiagram(entries_li_co_o)
    
    # Plot phase diagram
    plotter = PDPlotter(pd_li_co_o,show_unstable=0) #show_unstable key switches on/off showing unstable compounds
    plotter.show()

Get all the compounds of Li-Co-O system

In [ ]:
data = collections.defaultdict(list)

for e in pd_li_co_o.stable_entries:
    data["Materials ID"].append(e.entry_id)
    data["Composition"].append(e.composition.reduced_formula)
    data["Spacegroup"].append(e.structure.get_space_group_info())
    data["Ehull"].append(pd_li_co_o.get_e_above_hull(e))    
    data["Eform per at"].append(pd_li_co_o.get_form_energy_per_atom(e))    

df = DataFrame(data, columns=["Materials ID", "Spacegroup", "Composition", "Ehull","Eform per at"])
print(df)
df.to_csv('li-co-o_stable.csv') #write only stable compounds into li-co-o_stable.csv file

## 3. Phase diagram in chemical potential space

In [ ]:
#import necessary libraries
from pymatgen.analysis.phase_diagram import GrandPotentialPhaseDiagram, GrandPotPDEntry, PDEntry
from pymatgen.core.periodic_table import Element
from pymatgen.core.composition import Composition

**Simple way to construct such a phase diagram is using Materials project's data**

In [ ]:
#Use previously obtained entries from Materials Project
with MPRester(your_api_key) as mpr:

    # Obtain GGA, and GGA+U ComputedStructureEntry objects
    entries_li_co_o = mpr.get_entries_in_chemsys(elements=["Li", "Co", "O"], 
                                         additional_criteria={"thermo_types": ["GGA_GGA+U"]}) 

pd_li_co_o = PhaseDiagram(entries_li_co_o)

stable_entries=pd_li_co_o.stable_entries
PD = PhaseDiagram(stable_entries, )  
plot = PDPlotter(PD)

plt_mp = plot.get_chempot_range_map_plot([Element('Li'), Element('O')]) #phase diagram in Li-O chem pot space
filename = 'Li-Co-O_PD'
plt.savefig(filename, bbox_inches='tight')
plt.show()

**How to construct it using your compounds list and DFT calculated formation energies. You also can get rid of some phases from consideration**

In [ ]:
# Collect a dictionary of formation energies (from MP is ok) for interesting phases of considering system
# Note, pure components should have zero formation energy
# Own obtained formation energies can be used. Here we use data from Boev, A. O., et al. Applied Surface Science 537 (2021): 147750.
 
data_Li_Co_O = {
'Li'        : 0,
'O'         : 0,
'Co'        : 0,
'O2'        : 0,
'Co3O4'     : -1.394*7 , # mp-714961
'CoO2'      : -1.076*3 , # mp-1901
'CoO'       : -1.235*2 , # mp-715434
'LiCoO2'    : -1.780*4 , # mp-19149
'LiCo2O4'   : -1.516*7 , # mp-19149
'Li2O2'     : -1.760*4 , # mp-2340
'Li2O'      : -2.093*3 , # mp-2352
}

filename = 'chemical_phase_Li_Co_O.png'


entries = [] #create entries from dictionary
for key in data_Li_Co_O:
    entries.append(PDEntry(Composition(key), data_Li_Co_O[key]))

PD = PhaseDiagram(entries)  
plot = PDPlotter(PD)

plt_mp = plot.get_chempot_range_map_plot([Element('Li'), Element('O')]) #phase diagram in Li-O chem pot space

plt.savefig(filename,bbox_inches='tight')
plt.show()


## How to use it? Synthesis conditions via O chem pot (This information is just what you get familiar with)

In [ ]:
# thermochem is a special database containing information about entropy and enthalpy of main gases including O2
# !pip install thermochem
import math
from thermochem.burcat import Elementdb
from thermochem.janaf import Janafdb
# https://janaf.nist.gov/tables/O-029.html
# https://github.com/adelq/thermochem/blob/master/thermochem/janaf.py

# Define constants
atm = 101325 # Pa
R = 8.315 # J/mol/K
kJ_mol2eV = 1.0364e-2

# read Janaf database
db = Janafdb()
p_data = db.getphasedata(formula='O2', phase='ref')

In [ ]:
# The function to show dmu_O dependence on partial oxygen pressure

def dmu_p(T, unit = 'C'):
    #T in K
    import matplotlib.pyplot as plt
    import numpy as np

    if unit == 'C': T = T+273
    p_list = []
    dmu_list = []
    plt.figure(figsize = (8, 5))
    

    for p in range(1,100000, 100):
            RT = R*T
            p = p*0.00000987 # Pa to atm
            # data from JANAF
            dH = p_data.DeltaH(T)# Enthalpy in kJ/mol
            dS = (p_data.S(T) - p_data.S(0))# Entropy in J/mol/K
            dG_O2_T = dH - T*dS
            mu = dG_O2_T + RT*math.log(p)
            dmu = 0.5*(mu)*kJ_mol2eV/1000

            dmu_list.append(dmu)
            p_list.append(p)

    plt.plot(p_list, dmu_list, 'ko', alpha = 0.9, markersize = 0.7)
    plt.xlabel('p(O$_2$), atm', fontsize = 20, family='Serif')
    plt.ylabel('$\Delta\mu$(O), eV', fontsize = 20, family='Serif')
    # plt.xlim(0,50)
    plt.text(p/2, dmu*1.1,'T = {}$^\circ$ C'.format(T), fontsize = 20, family='Serif')
    # plt.ylim(33,1000)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.show()


In [ ]:
dmu_p(T=470, unit = 'С')

In [ ]:
# The function to show dmu_O dependence on synthesis temperature
def dmu_T(p):
    #p in atm
    import matplotlib.pyplot as plt
    import numpy as np
    import math
    T_list = []
    dmu_list = []
    plt.figure(figsize = (8, 5))

    for T in range(300,1273, 2):
            RT = R*T
            T_cels = T-273
            # data from JANAF
            dH = p_data.DeltaH(T)# Enthalpy in kJ/mol
            dS = (p_data.S(T) - p_data.S(0))# Entropy in J/mol/K
            dG_O2_T = dH - T*dS
            mu = dG_O2_T + RT*math.log(p)
            dmu = 0.5*(mu)*kJ_mol2eV/1000

            dmu_list.append(dmu)
            T_list.append(T_cels)

    plt.plot(T_list, dmu_list, 'ko', alpha = 0.9, markersize = 0.7)
    plt.xlabel('T, $^\circ$C', fontsize = 20, family='Serif')
    plt.ylabel('$\Delta\mu$(O), eV', fontsize = 20, family='Serif')
    plt.xlim(27,1000)
    plt.text(T_cels/2, dmu*0.5,'p(O$_2$) = {} atm'.format(p), fontsize = 20, family='Serif')
    # plt.ylim(33,1000)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    plt.show()



In [ ]:
dmu_T(p = 0.2) #oxygen partial pressure at 1 atm

## 4. Chemical reactions at interfaces

In [ ]:
from pymatgen.analysis.interface_reactions import InterfacialReactivity, GrandPotentialInterfacialReactivity
from pymatgen.analysis.phase_diagram import GrandPotentialPhaseDiagram, PhaseDiagram
from pymatgen.core import Composition, Element

%matplotlib inline

In [ ]:
# Chemical formulae for two solid reactants.
reactant1 = "LiCoO2"
reactant2 = "Li3PS4"

# Is the system open to an elemental reservoir?
grand = True

if grand:
    # Element in the elemental reservoir.
    open_el = "Co"
    # Relative chemical potential vs. pure substance. Must be non-positive.
    relative_mu = -1

In [ ]:
# Get the compositions of the reactants
comp1 = Composition(reactant1)
comp2 = Composition(reactant2)

# Gather all elements involved in the chemical system.
elements = [e.symbol for e in comp1.elements + comp2.elements]
if grand:
    elements.append(open_el)
elements = list(set(elements))  # Remove duplicates

# Get all entries in the chemical system
with MPRester(your_api_key) as mpr:
    entries = mpr.get_entries_in_chemsys(elements)

# Build a phase diagram using these entries.
pd = PhaseDiagram(entries)

# For an open system, include the grand potential phase diagram.
if grand:
    # Get the chemical potential of the pure subtance.
    mu = pd.get_transition_chempots(Element(open_el))[0]
    # Set the chemical potential in the elemental reservoir.
    chempots = {open_el: relative_mu + mu}
    # Build the grand potential phase diagram
    gpd = GrandPotentialPhaseDiagram(entries, chempots)
    # Create InterfacialReactivity object.
    interface = GrandPotentialInterfacialReactivity(
        comp1,
        comp2,
        gpd,
        norm=True,
        include_no_mixing_energy=True,
        pd_non_grand=pd,
        use_hull_energy=False,
    )
else:
    interface = InterfacialReactivity(
        comp1,
        comp2,
        pd,
        norm=True,
        include_no_mixing_energy=False,
        pd_non_grand=None,
        use_hull_energy=False,
    )

In [ ]:
# plot diagram
plt = interface.plot()
plt.show()

In [ ]:
from collections import OrderedDict
from pandas import DataFrame

critical_rxns = [
    OrderedDict(
        [
            ("Atomic fraction", round(ratio, 3)),
            ("Reaction equation", rxn),
            ("E$_{rxt}$ per mol equation (kJ/mol)", round(rxn_energy, 1)),
            ("E$_{rxt}$ per reactant atom (eV/atom)", round(reactivity, 3)),
        ]
    )
    for _, ratio, reactivity, rxn, rxn_energy in interface.get_kinks()
]
interface_reaction_table = DataFrame(critical_rxns)
interface_reaction_table

## 5. Pourbaix diagram

In [ ]:
!pip install mpcontribs-client

In [ ]:
# Import necessary tools from pymatgen
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter


In [ ]:
mpr = MPRester(your_api_key)


In [ ]:
# Get all pourbaix entries corresponding to the Mn-O-H chemical system.
entries = mpr.get_pourbaix_entries(["Mn"])


In [ ]:
# Construct the PourbaixDiagram object
pbx = PourbaixDiagram(entries)
# print(entries)

In [ ]:
plotter = PourbaixPlotter(pbx)
plotter.get_pourbaix_plot()


Compare obtained result with fig.34 in https://doi.org/10.59761/RCR5086